In [130]:
import pandas
import numpy as np
from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [131]:
#Reading data into a dataframe object
data = pandas.read_csv('train-missings.csv')
#Handling missing valuses using linear interpolation
data = data.interpolate()
data = data.fillna(0)
X = data.values[:,1:-1]
Y = data.values[:,-1]



In [132]:
#classes
set(Y)

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0}

In [128]:
repeat = True
useless_features = []
while repeat:
    scores = []
    for i in range(0,14):
        data = pandas.read_csv('train-missings.csv')
        #Handling missing valuses using linear interpolation
        data = data.interpolate()
        data = data.fillna(0)
        cols = list(range(1,14))
        for f in useless_features:
            cols.remove(f)
        if i > 0 and i in cols:
            cols.remove(i)
        X = data.values[:,cols]
        Y = data.values[:,-1]
        #scalling columns
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=1000).fit(X_train, y_train)
        scores.append(clf.score(X_test,y_test))
        max_index = scores.index(max(scores))
    if max_index > 0:  
        print("score is highest without feature p{}".format(max_index))
        print(scores[max_index])
        useless_features.append(max_index)
    else:
        print("useless features are {}".format(useless_features))
        repeat = False

score is highest without feature p5
0.6084656084656085
useless features are [5]


In [ ]:
cols = list(range(1,14))
for f in useless_features:
    cols.remove(f)
X = data.values[:,cols]

#scalling columns
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

Y = data.values[:,-1]
lb = preprocessing.LabelBinarizer()
lb.fit(Y)
Y = lb.transform(Y)

#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
neural_net = keras.models.Sequential()

neural_net.add(keras.layers.Dense(10, activation='sigmoid', input_shape=(X_train.shape[1],)))
#neural_net.add(keras.layers.Dense(10, activation='sigmoid'))
neural_net.add(keras.layers.Dense(7, activation='sigmoid'))
neural_net.compile(optimizer=tf.train.RMSPropOptimizer(0.001), loss='categorical_crossentropy' , metrics=['accuracy'])
neural_net.fit(X_train, y_train,epochs=30)

In [ ]:
neural_net.evaluate(X_test,y_test)